# Diseño y Vlsm 

# Documentación Técnica Final - VLSM Estricto (Criterio Aditivo)

## 1. Criterio de Diseño Actualizado
Siguiendo la instrucción docente (confirmada por el equipo), el cálculo de capacidad es **acumulativo**:
1.  **Base PCs:** Se calcula con la fórmula $200 + (10 \times N_{red})$. Esto cubre *solo* ordenadores de escritorio.
2.  **Carga Adicional:** Se suman **Laptops**, **Servidores** y **Dispositivos IoT** como extra a la base.

**Impacto:** Las VLANs con alta densidad de equipos mixtos (Control, Monitoreo, Wi-Fi) superan sistemáticamente el límite de 254 hosts, requiriendo máscaras **/23**.

---

## 2. Direccionamiento por Sede Detallado

### 📍 2.1 RÍO DE JANEIRO 

**Criterio:** Se respeta el orden estricto de la leyenda visual, la distribución física de los 34 dispositivos IoT y la carga de móviles.

**Fórmula:** $Base (200 + 10 \times Factor) + \text{Carga Extra}$

| Orden | Color (Leyenda) | VLAN Nombre | ID | Cálculo de Hosts Requeridos | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | **Verde** | **Admin** | 10 | $200 + 10(1) + 7 \text{ IoT} = \mathbf{217}$ | **/24** |
| 2 | **Rojo** | **Logistica** | 20 | $200 + 10(2) + 7 \text{ IoT} = \mathbf{227}$ | **/24** |
| 3 | **Azul** | **Prensa** | 30 | $200 + 10(3) + 5 \text{ IoT} = \mathbf{235}$ | **/24** |
| 4 | **Rosa** | **Seguridad** | 40 | $200 + 10(4) + 7 \text{ IoT} = \mathbf{247}$ | **/24** |
| 5 | **Magenta** | **Inalambrica**| 50 | $200 + 10(5) + 4 \text{ Hardw.} + \text{Móviles} > \mathbf{254}$ | **/23** |
| 6 | **Amarillo** | **Servidores**| 60 | $200 + 10(6) + 13 \text{ (5 Srv + 8 IoT)} = \mathbf{273}$ | **/23** |
| 7 | **Oliva** | **SSH** | 99 | $200 + 10(7) \text{ (Gestión)} = \mathbf{270}$ | **/23** |

*Se ajustó el orden de Inalámbrica y Servidores para coincidir con la leyenda visual. Ambas, junto con SSH, requieren /23.*

---

### 📝 Tabla de Direccionamiento IP (Sede Río)
**Bloque Base:** `10.2.64.0`

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/24** | `10.2.64.0` | `.64.2` a `.64.254` | `10.2.64.1` |
| **20** | Logistica | **/24** | `10.2.65.0` | `.65.2` a `.65.254` | `10.2.65.1` |
| **30** | Prensa | **/24** | `10.2.66.0` | `.66.2` a `.66.254` | `10.2.66.1` |
| **40** | Seguridad | **/24** | `10.2.67.0` | `.67.2` a `.67.254` | `10.2.67.1` |
| **50** | Inalambrica| **/23** | `10.2.68.0` | `.68.2` a `.69.254` | `10.2.68.1` |
| **60** | Servidores| **/23** | `10.2.70.0` | `.70.2` a `.71.254` | `10.2.70.1` |
| **99** | SSH | **/23** | `10.2.72.0` | `.72.2` a `.73.254` | `10.2.72.1` |
---

### 📍 2.2 SÃO PAULO

**Criterio:** Se respeta el orden de la leyenda visual. Aunque no hay una "VLAN Servidores" dedicada en la topología, la **VLAN Inalámbrica** contiene 1 Servidor físico y WLCs, lo que sumado a los móviles satura la red.

**Fórmula:** $Base (200 + 10 \times Factor) + \text{Carga Extra}$

| Orden | Color (Leyenda) | VLAN Nombre | ID | Cálculo de Hosts Requeridos | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | **Verde** | **Admin** | 10 | $200 + 10(1) = \mathbf{210}$ | **/24** |
| 2 | **Rojo** | **Logistica** | 20 | $200 + 10(2) = \mathbf{220}$ | **/24** |
| 3 | **Azul** | **Prensa** | 30 | $200 + 10(3) = \mathbf{230}$ | **/24** |
| 4 | **Rosa** | **Seguridad** | 40 | $200 + 10(4) = \mathbf{240}$ | **/24** |
| 5 | **Magenta** | **Inalambrica**| 50 | $200 + 10(5) + 4 \text{ (1 Srv + 3 WLC)} + \text{Móviles} > \mathbf{254}$ | **/23** |
| 6 | **Oliva** | **SSH** | 99 | $200 + 10(6) \text{ (Gestión)} = \mathbf{260}$ | **/23** |

*La VLAN Inalámbrica supera el límite de 254 IPs debido a la carga dinámica de móviles y hardware fijo, requiriendo /23. SSH también requiere /23 por factor incremental.*

---

### 📝 Tabla de Direccionamiento IP (Sede São Paulo)
**Bloque Base:** `10.2.74.0` *(Continúa inmediatamente después de Río)*

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/24** | `10.2.74.0` | `.74.2` a `.74.254` | `10.2.74.1` |
| **20** | Logistica | **/24** | `10.2.75.0` | `.75.2` a `.75.254` | `10.2.75.1` |
| **30** | Prensa | **/24** | `10.2.76.0` | `.76.2` a `.76.254` | `10.2.76.1` |
| **40** | Seguridad | **/24** | `10.2.77.0` | `.77.2` a `.77.254` | `10.2.77.1` |
| **50** | Inalambrica| **/23** | `10.2.78.0` | `.78.2` a `.79.254` | `10.2.78.1` |
| **99** | SSH | **/23** | `10.2.80.0` | `.80.2` a `.81.254` | `10.2.80.1` |

---

### 📍 2.3 SALVADOR (Operativa)

**Criterio:** El **Único Servidor** está alojado en la **VLAN Inalámbrica**, lo que obliga a esta última a usar /23.

**Distribución de Carga (26 IoTs Visuales + 1 Server):**

| Orden | Color (Foto) | VLAN Nombre | ID | Cálculo de Hosts Requeridos | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | **Verde** | **Admin** | 10 | $200 + 10(1) + \mathbf{7\ IoT} = \mathbf{217}$ | **/24** |
| 2 | **Rojo** | **Logistica** | 20 | $200 + 10(2) + \mathbf{7\ IoT} = \mathbf{227}$ | **/24** |
| 3 | **Azul** | **Prensa** | 30 | $200 + 10(3) + \mathbf{5\ IoT} = \mathbf{235}$ | **/24** |
| 4 | **Rosa** | **Seguridad** | 40 | $200 + 10(4) + \mathbf{7\ IoT} = \mathbf{247}$ | **/24** |
| 5 | **Magenta** | **Inalambrica**| 50 | $200 + 10(5) + \mathbf{1\ Srv} + \text{Móviles} > \mathbf{254}$ | **/23** |
| 6 | **Oliva** | **SSH** | 99 | $200 + 10(6) \text{ (Gestión)} = \mathbf{260}$ | **/23** |

*(Nota Crítica: La VLAN Seguridad llega a 247 hosts, dejando apenas 7 IPs libres. La VLAN Inalámbrica requiere /23 por alojar el Servidor local y la carga móvil).*

---

### 📝 Tabla de Direccionamiento IP (Sede Salvador)
**Continuidad:** Se ajusta para seguir inmediatamente después de São Paulo.
**Bloque Base:** `10.2.82.0`

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/24** | `10.2.82.0` | `.82.2` a `.82.254` | `10.2.82.1` |
| **20** | Logistica | **/24** | `10.2.83.0` | `.83.2` a `.83.254` | `10.2.83.1` |
| **30** | Prensa | **/24** | `10.2.84.0` | `.84.2` a `.84.254` | `10.2.84.1` |
| **40** | Seguridad | **/24** | `10.2.85.0` | `.85.2` a `.85.254` | `10.2.85.1` |
| **50** | **Inalambrica**| **/23** | `10.2.86.0` | `.86.2` a `.87.254` | `10.2.86.1` |
| **99** | **SSH** | **/23** | `10.2.88.0` | `.88.2` a `.89.254` | `10.2.88.1` |

---

### 📍 2.4 RECIFE (Soporte)

**Criterio:** Sede administrativa sin carga IoT. Se respeta la fórmula incremental. La presencia del **Servidor 3** y los controladores en la VLAN Inalámbrica, sumado a los móviles, fuerza el uso de /23 en ese segmento.

**Distribución de Carga (0 IoTs + 1 Server):**

| Orden | Color (Foto) | VLAN Nombre | ID | Cálculo de Hosts Requeridos | Máscara |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 1 | **Verde** | **Admin** | 10 | $200 + 10(1) = \mathbf{210}$ | **/24** |
| 2 | **Rojo** | **Logistica** | 20 | $200 + 10(2) = \mathbf{220}$ | **/24** |
| 3 | **Azul** | **Prensa** | 30 | $200 + 10(3) = \mathbf{230}$ | **/24** |
| 4 | **Rosa** | **Seguridad** | 40 | $200 + 10(4) = \mathbf{240}$ | **/24** |
| 5 | **Magenta** | **Inalambrica**| 50 | $200 + 10(5) + \mathbf{1\ Srv} + \text{Móviles} > \mathbf{254}$ | **/23** |
| 6 | **Oliva** | **SSH** | 99 | $200 + 10(6) \text{ (Gestión)} = \mathbf{260}$ | **/23** |

*(Nota: Las salas operativas no cuentan con dispositivos IoT visibles, por lo que se mantienen en /24. La red Inalámbrica y SSH requieren /23 por carga base y móviles).*

---

### 📝 Tabla de Direccionamiento IP (Sede Recife)
**Continuidad:** Inicia inmediatamente después del bloque de Salvador (`10.2.89.255`).
**Bloque Base:** `10.2.90.0`

| VLAN ID | Nombre | Máscara | Dirección de Red | Rango de IPs Útiles | Gateway (Router) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **10** | Admin | **/24** | `10.2.90.0` | `.90.2` a `.90.254` | `10.2.90.1` |
| **20** | Logistica | **/24** | `10.2.91.0` | `.91.2` a `.91.254` | `10.2.91.1` |
| **30** | Prensa | **/24** | `10.2.92.0` | `.92.2` a `.92.254` | `10.2.92.1` |
| **40** | Seguridad | **/24** | `10.2.93.0` | `.93.2` a `.93.254` | `10.2.93.1` |
| **50** | **Inalambrica**| **/23** | `10.2.94.0` | `.94.2` a `.95.254` | `10.2.94.1` |
| **99** | **SSH** | **/23** | `10.2.96.0` | `.96.2` a `.97.254` | `10.2.96.1` |

---

## 3. Enlaces WAN (Backbone)
**Validación:** 5 enlaces serie. La asignación DCE (lado con reloj) se basa en el diagrama de topología visual.

**Bloque Asignado:** `10.2.98.0`

**Máscara:** `/30` (255.255.255.252)

| Enlace | Router (DCE - Reloj) | IP | Router (DTE) | IP | Dirección de Red |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **Río -> SP** | Río `Se0/3/0` | **`.98.1`** | SP `Se0/3/0` | **`.98.2`** | `10.2.98.0/30` |
| **Río -> Sal** | Río `Se0/2/0` | **`.98.5`** | Sal `Se0/3/0` | **`.98.6`** | `10.2.98.4/30` |
| **Río -> Rec** | Río `Se0/3/1` | **`.98.9`** | Rec `Se0/3/0` | **`.98.10`**| `10.2.98.8/30` |
| **SP -> Rec** | SP `Se0/3/1` | **`.98.13`**| Rec `Se0/3/1` | **`.98.14`**| `10.2.98.12/30` |
| **Sal -> Rec** | Sal `Se0/3/1` | **`.98.17`**| Rec `Se0/2/0` | **`.98.18`**| `10.2.98.16/30` |